In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

def load_dataset(train_folder, test_folder, transform):
    train_dataset = datasets.ImageFolder(train_folder, transform=transform)
    test_dataset = datasets.ImageFolder(test_folder, transform=transform)
    return train_dataset, test_dataset

def train(model, criterion, optimizer, train_loader, val_loader,test_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}")
                running_loss = 0.0

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                if torch.cuda.is_available():
                    images, labels = images.cuda(), labels.cuda()
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        # Calculate average validation loss and accuracy for the epoch
        val_loss /= len(val_loader)
        val_accuracy = correct / total * 100
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
    with torch.no_grad():
        for images, labels in test_loader:
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()

            # Forward pass
            outputs = model(images)

            # Calculate loss
            test_loss = criterion(outputs, labels)

            # Update validation loss
            test_loss += test_loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        # Calculate average validation loss and accuracy for the epoch
        test_loss /= len(test_loader)
        test_accuracy = correct / total * 100

#         wandb.log({'Val_Loss': val_loss, 'val_accuracy': val_accuracy})
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    
    
# Define the transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_folder = '/kaggle/input/dataset-cs6910-a2/inaturalist_12K/train'
test_folder = '/kaggle/input/dataset-cs6910-a2/inaturalist_12K/val'
train_dataset, test_dataset = load_dataset(train_folder, test_folder, transform)

# Split train dataset into train and validation sets
train_indices, val_indices = train_test_split(list(range(len(train_dataset))), test_size=0.2, shuffle=True, stratify=train_dataset.targets)
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

# Create DataLoader for train and validation datasets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
# # Load pre-trained ResNet50 model
# model = models.resnet50(weights='imagenet')

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))

# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

# Train the model
num_epochs = 10
train(model, criterion, optimizer, train_loader, val_loader, test_loader, num_epochs)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 43.9MB/s]


Epoch [1/10], Step [100/250], Loss: 1.3928
Epoch [1/10], Step [200/250], Loss: 0.9851
Validation Loss: 0.9032, Validation Accuracy: 71.25%
Epoch [2/10], Step [100/250], Loss: 0.7915
Epoch [2/10], Step [200/250], Loss: 0.8012
Validation Loss: 0.8151, Validation Accuracy: 73.00%
Epoch [3/10], Step [100/250], Loss: 0.7931
Epoch [3/10], Step [200/250], Loss: 0.7667
Validation Loss: 0.8202, Validation Accuracy: 74.00%
Epoch [4/10], Step [100/250], Loss: 0.7399
Epoch [4/10], Step [200/250], Loss: 0.7539
Validation Loss: 0.7963, Validation Accuracy: 74.65%
Epoch [5/10], Step [100/250], Loss: 0.7056
Epoch [5/10], Step [200/250], Loss: 0.7503
Validation Loss: 0.7910, Validation Accuracy: 74.85%
Epoch [6/10], Step [100/250], Loss: 0.7206
Epoch [6/10], Step [200/250], Loss: 0.7288
Validation Loss: 0.7936, Validation Accuracy: 74.65%
Epoch [7/10], Step [100/250], Loss: 0.6656
Epoch [7/10], Step [200/250], Loss: 0.7027
Validation Loss: 0.8178, Validation Accuracy: 73.65%
Epoch [8/10], Step [100/250